In [3]:
#!/usr/bin/env python3
import pandas as pd
from pathlib import Path
from tqdm import tqdm

INPUT_CSV = Path("/lisc/data/scratch/becogbio/juarez/test_thesis/Combined_predict_vp_individual/combined_predictions_with_video_name.csv")
OUTPUT_CSV = INPUT_CSV.with_name("video_level_majority_vote.csv")

def majority_vote(series: pd.Series):
    """
    Returns the most frequent value in a Pandas Series.
    In case of ties, the first encountered most-frequent value is returned.
    """
    return series.value_counts().idxmax()

def main():
    # Load CSV
    df = pd.read_csv(INPUT_CSV, dtype=str)

    # Sanity check
    for col in ["Video Name", "Inddividual_prediction"]:
        if col not in df.columns:
            raise ValueError(f"Missing column '{col}' in input CSV.")

    # Prepare results list
    results = []

    # Group by Video Name manually to use tqdm
    grouped = df.groupby("Video Name")
    for video_name, group in tqdm(grouped, desc="Processing videos", unit="video"):
        winner = majority_vote(group["Inddividual_prediction"])
        results.append({"Video Name": video_name, "Majority_Individual_prediction": winner})

    # Create final DataFrame
    df_video = pd.DataFrame(results)

    # Save
    df_video.to_csv(OUTPUT_CSV, index=False)
    print(f"[OK] Video-level predictions written to: {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


Processing videos: 100%|██████████| 1808/1808 [00:00<00:00, 3427.06video/s]

[OK] Video-level predictions written to: /lisc/data/scratch/becogbio/juarez/test_thesis/Combined_predict_vp_individual/video_level_majority_vote.csv


# Per video evaluation

In [4]:
#!/usr/bin/env python3
import pandas as pd
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns

VIDEO_PRED_CSV = Path("/lisc/data/scratch/becogbio/juarez/test_thesis/Combined_predict_vp_individual/video_level_majority_vote.csv")
META_CSV = Path("/lisc/data/scratch/becogbio/juarez/test_thesis/0_raw_videos/test_videos/test_videos_metadata.csv")
OUTPUT_EVAL = Path("/lisc/data/scratch/becogbio/juarez/test_thesis/2_individual_classifier/test_on_test_set/per_video/video_level_evaluation.csv")
OUTPUT_SUMMARY = Path("/lisc/data/scratch/becogbio/juarez/test_thesis/2_individual_classifier/test_on_test_set/per_video/video_level_summary_per_bird.csv")
OUTPUT_CM = Path("/lisc/data/scratch/becogbio/juarez/test_thesis/2_individual_classifier/test_on_test_set/per_video/video_level_confusion_matrix.png")

def main():
    # Load predictions
    df_pred = pd.read_csv(VIDEO_PRED_CSV, dtype=str)
    if not {"Video Name", "Majority_Individual_prediction"} <= set(df_pred.columns):
        raise ValueError("Predictions CSV must have 'Video Name' and 'Majority_Individual_prediction' columns.")

    # Load metadata
    df_meta = pd.read_csv(META_CSV, dtype=str)
    if not {"Video_ID", "Bird_ID"} <= set(df_meta.columns):
        raise ValueError("Metadata CSV must have 'Video_ID' and 'Bird_ID' columns.")

    # Merge predictions with ground truth
    df_eval = pd.merge(df_pred, df_meta, left_on="Video Name", right_on="Video_ID", how="inner")
    if df_eval.empty:
        raise ValueError("No matches found between predictions and metadata!")

    # Evaluate
    y_true = df_eval["Bird_ID"]
    y_pred = df_eval["Majority_Individual_prediction"]

    # --- Classification report ---
    report = classification_report(y_true, y_pred, digits=3)
    print("\n=== Classification Report ===")
    print(report)

    # --- Per-bird summary table ---
    summary_records = []
    labels = sorted(df_eval["Bird_ID"].unique())
    p, r, f1, support = precision_recall_fscore_support(y_true, y_pred, labels=labels, zero_division=0)

    for bird, prec, rec, f1s, sup in zip(labels, p, r, f1, support):
        n_correct = ((df_eval["Bird_ID"] == bird) & (df_eval["Majority_Individual_prediction"] == bird)).sum()
        summary_records.append({
            "Bird_ID": bird,
            "n_videos": sup,
            "n_correct": n_correct,
            "accuracy": n_correct / sup if sup > 0 else 0.0,
            "precision": prec,
            "recall": rec,
            "f1_score": f1s,
        })

    df_summary = pd.DataFrame(summary_records)
    df_summary.to_csv(OUTPUT_SUMMARY, index=False)
    print(f"[OK] Per-bird summary written to: {OUTPUT_SUMMARY}")

    # Save merged results for traceability
    df_eval.to_csv(OUTPUT_EVAL, index=False)
    print(f"[OK] Evaluation file written to: {OUTPUT_EVAL}")

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greys", xticklabels=labels, yticklabels=labels)
    plt.xlabel("Predicted")
    plt.ylabel("Ground Truth")
    plt.title("Video-level Confusion Matrix")
    plt.tight_layout()
    plt.savefig(OUTPUT_CM)
    plt.close()
    print(f"[OK] Confusion matrix saved to: {OUTPUT_CM}")

if __name__ == "__main__":
    main()



=== Classification Report ===
              precision    recall  f1-score   support

     BNU-RPM      0.986     0.973     0.980       149
     BNY-RPM      0.918     0.867     0.891        90
     BRG-YOM      0.954     0.989     0.971       272
     BRK-NOM      0.800     0.889     0.842        27
     EYB-RPM      0.970     0.987     0.978       226
     GBM-ORY      0.958     0.944     0.951       144
     GBY-ORM      0.924     0.964     0.944       165
     OEB-RPM      0.959     0.922     0.940        51
     OGY-BRM      0.986     0.892     0.937       158
     ORB-UYM      0.828     0.972     0.895       109
     OUB-RPM      0.902     0.846     0.873        65
     OYR-BGM      0.933     0.906     0.920       139
     RGY-BOM      0.681     0.542     0.604        59
     RYO-BGM      0.872     0.965     0.916        85
      YM-OBR      0.925     0.949     0.937        39
     YRU-POM      1.000     0.767     0.868        30

    accuracy                          0.931      

# Per video evaluation PER VIEWPOINT

In [10]:
import pandas as pd

# --- PATHS ---
INPUT_CSV = "/lisc/data/scratch/becogbio/juarez/test_thesis/Combined_predict_vp_individual/combined_predictions_with_video_name.csv"
OUTPUT_CSV = "/lisc/data/scratch/becogbio/juarez/test_thesis/2_individual_classifier/test_on_test_set/per_video_per_viewpoint/video_level_majority_per_viewpointt.csv"

# --- LOAD DATA ---
df = pd.read_csv(INPUT_CSV, dtype=str)

# --- MAJORITY VOTING FUNCTION ---
def majority_vote(labels):
    """Return the most common label in a group."""
    return labels.value_counts().idxmax()

# --- PER-VIDEO, PER-VIEWPOINT MAJORITY ---
grouped = df.groupby(["Video Name", "Viewpoint_prediction"])["Inddividual_prediction"]
majority = grouped.apply(majority_vote).reset_index()

# --- PIVOT TO GET ONE COLUMN PER VIEWPOINT ---
result = majority.pivot(
    index="Video Name",
    columns="Viewpoint_prediction",
    values="Inddividual_prediction"
).reset_index()

# --- ADD OVERALL MAJORITY (ALL VIEWPOINTS) ---
overall_majority = df.groupby("Video Name")["Inddividual_prediction"].apply(majority_vote)
result["all_viewpoints"] = result["Video Name"].map(overall_majority)

# --- SAVE OUTPUT ---
result.to_csv(OUTPUT_CSV, index=False)
print(f"[OK] Per-video (per-viewpoint + overall) majority predictions saved to: {OUTPUT_CSV}")

[OK] Per-video (per-viewpoint + overall) majority predictions saved to: /lisc/data/scratch/becogbio/juarez/test_thesis/2_individual_classifier/test_on_test_set/per_video_per_viewpoint/video_level_majority_per_viewpointt.csv


# Per viewpoint majority voting

In [15]:
#!/usr/bin/env python3
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

# --- PATHS ---
PRED_CSV = "/lisc/data/scratch/becogbio/juarez/test_thesis/Combined_predict_vp_individual/video_level_majority_per_viewpointt.csv"
META_CSV = "/lisc/data/scratch/becogbio/juarez/test_thesis/0_raw_videos/test_videos/test_videos_metadata.csv"
OUTPUT_CSV = "/lisc/data/scratch/becogbio/juarez/test_thesis/Combined_predict_vp_individual/f1_scores_per_bird_per_viewpoint.csv"

# --- LOAD ---
df_pred = pd.read_csv(PRED_CSV, dtype=str)
df_meta = pd.read_csv(META_CSV, dtype=str)
df = pd.merge(df_pred, df_meta, left_on="Video Name", right_on="Video_ID", how="inner")
if df.empty:
    raise ValueError("No matching videos between predictions and metadata!")

viewpoint_cols = [c for c in df_pred.columns if c != "Video Name"]
birds = sorted(df["Bird_ID"].unique())
summary = pd.DataFrame({"Bird_ID": birds})

for vp in viewpoint_cols:
    sub = df.dropna(subset=[vp])
    coverage = len(sub) / len(df)
    print(f"[INFO] Viewpoint '{vp}': using {len(sub)}/{len(df)} videos ({coverage:.1%} coverage)")

    y_true = sub["Bird_ID"].astype(str)
    y_pred = sub[vp].astype(str)

    _, _, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, labels=birds, zero_division=0
    )

    summary[vp] = f1  # add as a new column

summary.to_csv(OUTPUT_CSV, index=False)
print(f"[OK] Saved per-bird F1-scores per viewpoint to: {OUTPUT_CSV}")


[INFO] Viewpoint 'back': using 1265/1808 videos (70.0% coverage)
[INFO] Viewpoint 'front': using 1076/1808 videos (59.5% coverage)
[INFO] Viewpoint 'left_side': using 1062/1808 videos (58.7% coverage)
[INFO] Viewpoint 'right_side': using 961/1808 videos (53.2% coverage)
[INFO] Viewpoint 'all_viewpoints': using 1808/1808 videos (100.0% coverage)
[OK] Saved per-bird F1-scores per viewpoint to: /lisc/data/scratch/becogbio/juarez/test_thesis/Combined_predict_vp_individual/f1_scores_per_bird_per_viewpoint.csv
